In [36]:
import GPP
import numpy as np
import random
import sympy as sym

In [299]:
import importlib
importlib.reload(GPP)

<module 'GPP' from 'C:\\Users\\nacho\\OneDrive\\Infor\\5 (Sicue)\\TFG\\Implementation\\GPP.py'>

# BARNES LAPLACIA FOR UNBALANCED BIPARTITION

In [2]:
def random_johnson_graph (n, p):
    G = []
    for i in range(n+1):
        G.append([])
    G[0] = [n, 0]
    for i in range(1, n+1):
        for j in range(i+1, n+1):
            if random.random() < p:
                G[0][1] += 1
                G[i].append(j)
                G[j].append(i)
    return G

In [3]:
def graph_to_file (G, filename):
    gstr = ''
    for line in G:
        for number in line:
            gstr += str(number) + ' '
        gstr += '\n'
    
    with open(filename, 'w') as f:
        f.write(gstr)
    return

In [19]:
def all_solutions_unbalanced (n):
    
    sols = []
    for i in range (2**n):
        s = [int(d) for d in str(format(i, '0{}b'.format(n)))]
        if (s.count(0)>0 and s.count(1)>0):
            sols.append(s)
    
    return sols

In [23]:
def are_neighbors_hamming (x, y):
    neighbor = False
    diff = (x - y).tolist()
    # diff list has as many 0s as equal positions have 'x' and 'y'
    if ((len(diff) - diff.count(0)) ==  1):
        neighbor = True

    return neighbor

In [75]:
n = 4
k = 2
reps = 10

In [76]:
for i in range(reps):
    print('Checking graph ' + str(i))
    
    # Generate a random graph and saved it
    p = random.random()
    G = random_johnson_graph(n, p)
    filename = 'random_graphs/' + str(i) + '-G' + str(n) + '.' + str(round(p, 3)).split('.')[-1] + '.graph'
    graph_to_file(G, filename)
    gpp = GPP.GPP(filename, k)
    
    # Search space
    X = np.array(all_solutions_unbalanced(n))
    
    # Objective function
    ff = np.array([gpp.f(x) for x in X])
    
    # Adjacency Matrix
    A = np.zeros([X.shape[0], X.shape[0]], dtype='int')
    for i in range(X.shape[0]):
        for j in range (X.shape[0]):
            if (are_neighbors_hamming(X[i], X[j])):
                A[i][j] = 1

    # Degree Matrix
    D = np.zeros([X.shape[0], X.shape[0]], dtype='int')
    for i in range(A.shape[0]):
        D[i][i] = np.count_nonzero(A[i])

    # Barnes Laplacian
    Dinv = np.linalg.inv(D)
    I = np.identity(A.shape[0])
    L = I - np.matmul(Dinv, A)
    
    # Compute eigenvalues
    eigvals, eigvecs = np.linalg.eig(L)
    
    # Check if the system has a solution
    solvable = False
    for lamb in eigvals:
        a = sym.Symbol('a')
        izq = L @ (ff-a)
        der = lamb * (ff-a)
        sol = sym.solve(izq - der, a)
        if sol:
            solvable = True
            solution = sol
    if solvable:
        print('   SOLUTION FOUND: ' +str(solution))

Checking graph 0
   SOLUTION FOUND: {a: 3.50000000000000}
Checking graph 1
   SOLUTION FOUND: {a: 0.0}
Checking graph 2
Checking graph 3
Checking graph 4
   SOLUTION FOUND: {a: 3.50000000000000}
Checking graph 5
Checking graph 6
Checking graph 7
Checking graph 8
Checking graph 9
